## Libraries

In [1]:
# importing necessary libraries
import pandas as pd  # data analysis
import numpy as np  # mathematic evaluations
import pycountry # ISO database for countries

## Imports

In [2]:
# Import the CSV files into dfs:
df_class = pd.read_csv('C:/Users/lluis/Desktop/Documents/IronHack/Final_Project/data/raw/1-world_economic_classifications_v2.csv')
df_gdppc_imf = pd.read_csv('C:/Users/lluis/Desktop/Documents/IronHack/Final_Project/data/raw/2-gdp_per_capita_imf.csv', encoding='latin1')
df_gdp_national = pd.read_csv('C:/Users/lluis/Desktop/Documents/IronHack/Final_Project/data/raw/3-national_gdp_wb.csv')
df_exchange_rate = pd.read_csv('C:/Users/lluis/Desktop/Documents/IronHack/Final_Project/data/raw/4-nasdaq_real_time_price.csv')

## Cleaning df_class

In [3]:
df_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   country_name      205 non-null    object 
 1   un_class_2014     167 non-null    object 
 2   imf_class_2023    193 non-null    object 
 3   g7                205 non-null    object 
 4   eu_member         205 non-null    object 
 5   fuel_exp_country  205 non-null    object 
 6   wealth_rank       178 non-null    float64
 7   gdp_ppp_2022      175 non-null    object 
 8   gdp_pc_2022       176 non-null    object 
dtypes: float64(1), object(8)
memory usage: 14.5+ KB


We are applying the correct column type for each column

In [4]:
# We convert the 'g7', 'eu_member', and 'fuel_exp_country' columns to boolean type by mapping 'Yes' to 1 and 'No' to 0
mapping_dict = {'Yes': 1, 'No': 0}

# Apply the mapping to the columns
df_class['g7'] = df_class['g7'].map(mapping_dict)
df_class['eu_member'] = df_class['eu_member'].map(mapping_dict)
df_class['fuel_exp_country'] = df_class['fuel_exp_country'].map(mapping_dict)

# We change these columns to boolean type:
df_class['g7'] = df_class['g7'].astype(bool)
df_class['eu_member'] = df_class['eu_member'].astype(bool)
df_class['fuel_exp_country'] = df_class['fuel_exp_country'].astype(bool)

Since df_class will be our main DataFrame, we will add the ISO country codes (both 2-digit and 3-digit) to facilitate later plotting and visualization.

In [5]:
# Some names are incorrect so we will try to change the names to be able to locate the correct ISO country codes.
# We define a dictionary with old country names as keys and new country names as values
correct_country_names = {
    "Macao SAR": "Macao",
    "Turkey": "Türkiye",
    "Russia": "Russian Federation",
    "St. Kitts and Nevis": "Saint Kitts And Nevis",
    "Macedonia": "North Macedonia",
    "Azerbijan": "Azerbaijan",
    "St. Lucia": "Saint Lucia",
    "Equitorial Guinea": "Guinea",
    "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Palestine": "Palestine, State of",
    "Ivory Coast": "Côte d'Ivoire",
    "Sao Tome and Prinicipe": "Sao Tome and Principe",
    "Micronesia": "Micronesia, Federated States of",
    "Democratic Republic of Congo": "Congo, The Democratic Republic of the",
    "Channel Islands": "Jersey",
    "Lichtenstein": "Liechtenstein",
}

# Replace old country names with new ones
df_class['country_name'] = df_class['country_name'].replace(correct_country_names)

In [6]:
# df_class will be our main DataFrame. We will use column "country_name" to extract ISO country codes.

# Function to get ISO country codes
def get_iso_codes(country_name):
    try:
        country = pycountry.countries.lookup(country_name)
        return country.alpha_2, country.alpha_3
    except LookupError:
        return None, None

# Apply the function to add ISO codes to the DataFrame
df_class[['ISO2', 'ISO3']] = df_class['country_name'].apply(lambda x: pd.Series(get_iso_codes(x)))

In [7]:
# We check if there are some countries without ISO Code
df_class['ISO2'].isna().sum()

0

In [8]:
# Now we drop gdp_ppp_2022 and gdp_pc_2022 columns from df_class, because we want to use the data provided by the World Bank

df_class = df_class.drop(['gdp_ppp_2022', 'gdp_pc_2022'], axis=1)

In [9]:
# We add GDP per capita from df_gdppc_imf to df_class

df_merged = df_class.merge(df_gdppc_imf, left_on='country_name', right_on='Country', how='inner')

In [10]:
# We drop column 'Country' because its redundant

df_merged = df_merged.drop('Country', axis=1)

In [11]:
# We replace the fields where there is no information for NaN

df_merged = df_merged.replace('no data', np.nan)

## Cleaning df_exchange_rate

In [12]:
df_exchange_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7888 entries, 0 to 7887
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       7888 non-null   object 
 1   Open       7888 non-null   float64
 2   High       7888 non-null   float64
 3   Low        7888 non-null   float64
 4   Close      7888 non-null   float64
 5   Adj Close  7888 non-null   float64
 6   Volume     7888 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 431.5+ KB


In [13]:
# We convert the date to datetime
df_exchange_rate['Date'] = pd.to_datetime(df_exchange_rate['Date'])

In [14]:
# We check for NaNs
df_exchange_rate.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

## Export Cleaned Data

In [15]:
# We export to csv the dfs we are going to use to cleaned data folder

df_merged.to_csv(r'C:/Users/lluis/Desktop/Documents/IronHack/Final_Project/data/cleaned/df_merged.csv', index=False)
df_exchange_rate.to_csv(r'C:/Users/lluis/Desktop/Documents/IronHack/Final_Project/data/cleaned/df_exchange_rate.csv', index=False)